In [20]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [21]:
with open("reviews_Books_5.json.002","r") as data_file:
    raw_data=data_file.readlines() 

In [22]:
json_list=[]

for i in range(len(raw_data)):
    try:
        json_data=json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i], i)

n he claims not to be.  Sarcasm and snide remarks permeate the book often serving to distract from the valid points it has to offer.Bernard Goldberg's years as an insider give him a valuable perspective about the current media climate, however this book is more of a dull whine than a clarion call.  I look forward to seeing more on this subject from such a well informed insider as Mr. Goldberg after he has finished venting his anger.", "overall": 4.0, "summary": "The end of the vast left wing conspiracy.", "unixReviewTime": 1012867200, "reviewTime": "02 5, 2002"}
 0
{"reviewerID": "A3CFM008T8ZZG5", "asin": "0060888741", "reviewerName": "Terry D. Janes \"TheVikingsWorld.com\"", "helpful": [0, 1], "reviewText": "Many people rave about Bernard Cornw 84965


In [23]:
review_data = []
x_data = []
y_data = []
for i in range(len(json_list)):
    y_data.append((int(json_list[i]["overall"]),))
    x_data.append(json_list[i]["reviewText"])

In [24]:
y_data = np.array(MultiLabelBinarizer().fit_transform(y_data))

In [25]:
def pos_tagging(document):
    result = nltk.pos_tag( nltk.word_tokenize(document))    
    last_result = ['/'.join(t) for t in result]
    return last_result
# 품사 tagging

In [ ]:
tagging_texts = [pos_tagging(text) for text in x_data]

In [13]:
choose_tagging_texts = []
for i in range(len(tagging_texts)):
    choose_tagging_texts.append([])
    for j in tagging_texts[i]:
        if j.split("/")[1] =="NN":
            choose_tagging_texts[i].append(j)
# NN만 추출

In [14]:
data_handle = []
for i in choose_tagging_texts:
    data_handle.append(" ".join(i))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1)
D = vectorizer.fit_transform(data_handle)

In [16]:
D = D.toarray()

MemoryError: 

In [17]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [11, 95])

w1 = tf.Variable(tf.truncated_normal([95, 50], stddev = 0.1))
# stddev 정규화 해주기/ 50 : layer 늘렸다 줄이기 
b1 = tf.Variable(tf.truncated_normal([50]))
# 보정값
w2 = tf.Variable(tf.truncated_normal([50, 5], stddev = 0.1))
b2 = tf.Variable(tf.truncated_normal([5]))

output =  tf.sigmoid(tf.matmul(x, w1) + b1)
hypothesis = tf.sigmoid(tf.matmul(output,w2)+b2)

cost = -tf.reduce_sum(y_data * tf.log(tf.clip_by_value(hypothesis,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.01).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(200):
    sess.run(train_step, feed_dict={x:D})
    if i % 100 == 0:
        print(i, sess.run(cost, feed_dict={x:D}))
        
# 뉴럴넷

ValueError: Dimensions must be equal, but are 84964 and 11 for 'mul' (op: 'Mul') with input shapes: [84964,5], [11,5].

In [ ]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y_data,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: D}))
# 정확도 측정

In [ ]:
print(sess.run(hypothesis, feed_dict={x: D}))
# 결과값

In [ ]:
print(sess.run(tf.argmax(hypothesis, 1), feed_dict={x: D}))
# 결과값 중에 높은 인덱스 가져오기